In [1]:
import pandas as pd
import numpy as np
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from keras.models import Model
from tensorflow.keras.optimizers import Adam
from keras.utils import np_utils

In [2]:
# Load the MNIST dataset
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

In [3]:
# Split the data into features and labels
X_train = train_df.iloc[:,1:].values
y_train = train_df.iloc[:,0].values
X_test = test_df.values

In [4]:
# Preprocess the data
X_train = X_train.reshape(-1, 28, 28, 1) / 255.0
X_test = X_test.reshape(-1, 28, 28, 1) / 255.0
y_train = np_utils.to_categorical(y_train, 10)


In [5]:
# Create the CNN model
cnn_input = Input(shape=(28, 28, 1))
cnn = Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same')(cnn_input)
cnn = Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same')(cnn)
cnn = MaxPooling2D(pool_size=(2, 2))(cnn)
cnn = Dropout(0.25)(cnn)
cnn = Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same')(cnn)
cnn = Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same')(cnn)
cnn = MaxPooling2D(pool_size=(2, 2))(cnn)
cnn = Dropout(0.25)(cnn)
cnn = Flatten()(cnn)

In [6]:
# Connect the output of CNN model to the input of DNN model
dnn_input = cnn

In [7]:
# Create the DNN model
dnn = Dense(512, activation='relu')(dnn_input)
dnn = Dropout(0.5)(dnn)
dnn = Dense(256, activation='relu')(dnn)
dnn = Dropout(0.5)(dnn)
dnn = Dense(10, activation='softmax')(dnn)

In [8]:
# Combine the CNN and DNN models
model = Model(inputs=cnn_input, outputs=dnn)

In [9]:
adam = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False)
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=20, batch_size=128, validation_split=0.1)

Epoch 1/20
296/296 [==============================] - 7s 11ms/step - loss: 0.3546 - accuracy: 0.8840 - val_loss: 0.0629 - val_accuracy: 0.9781
Epoch 2/20
296/296 [==============================] - 3s 9ms/step - loss: 0.0958 - accuracy: 0.9720 - val_loss: 0.0434 - val_accuracy: 0.9840
Epoch 3/20
296/296 [==============================] - 3s 9ms/step - loss: 0.0677 - accuracy: 0.9805 - val_loss: 0.0369 - val_accuracy: 0.9850
Epoch 4/20
296/296 [==============================] - 3s 9ms/step - loss: 0.0548 - accuracy: 0.9840 - val_loss: 0.0375 - val_accuracy: 0.9883
Epoch 5/20
296/296 [==============================] - 3s 9ms/step - loss: 0.0442 - accuracy: 0.9872 - val_loss: 0.0439 - val_accuracy: 0.9869
Epoch 6/20
296/296 [==============================] - 3s 9ms/step - loss: 0.0434 - accuracy: 0.9868 - val_loss: 0.0244 - val_accuracy: 0.9921
Epoch 7/20
296/296 [==============================] - 3s 9ms/step - loss: 0.0344 - accuracy: 0.9901 - val_loss: 0.0237 - val_accuracy: 0.9912
Epoch

In [10]:
y_pred = model.predict(X_test)
y_pred = np.argmax(y_pred, axis=1)
y_pred=pd.Series(y_pred,name="Label")

In [11]:
submission = pd.concat([pd.Series(range(1,28001),name = "ImageId"),y_pred],axis=1)
submission.to_csv("submission.csv",index=False)